In [ ]:
# timeit
# importing timeit
import timeit
code_to_test =  """
# Student Name : Georg Stotzingen
# Cohort       : 1 Lombard

# Note: You are only allowed to submit ONE final model for this assignment.


################################################################################
# Import Packages
################################################################################


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import sklearn.linear_model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error


################################################################################
# Load Data
################################################################################

# use this space to load the original dataset
# MAKE SURE TO SAVE THE ORIGINAL FILE AS original_df
# Example: original_df = pd.read_excel('Apprentice Chef Dataset.xlsx')

original_df = pd.read_excel('C:/Users/Georg/Desktop/Hult/MSBA/Machine Learning/Case/Apprentice_Chef_Dataset.xlsx')




################################################################################
# Feature Engineering and (optional) Dataset Standardization
################################################################################

# use this space for all of the feature engineering that is required for your
# final model

# if your final model requires dataset standardization, do this here as well

# Making a copy with shorter name to protect original_df.
df = original_df.copy()

# Lower casing for ease of use.
df.columns = map (str.lower, df.columns)

# Creating a missing values column. 
for col in df:
    if df[col].isnull().astype(int).sum() > 0:
        df['m_'+col] = df[col].isnull().astype(int)

#setting outlier thresholds
revenue_hi                          = 5500
total_meals_ordered_hi              = 220
total_meals_ordered_lo              = 16
unique_meals_purch_hi               = 9
contacts_w_customer_service_hi      = 13
contacts_w_customer_service_lo      = 3
product_categories_viewed_hi        = 10
product_categories_viewed_lo        = 1
avg_time_per_site_visit_hi          = 260
avg_time_per_site_visit_lo          = 20
cancellations_before_noon_hi        = 5
cancellations_after_noon_hi         = 1.5
mobile_logins_lo                    = 5
pc_logins_hi                        = 2
weekly_plan_hi                      = 20
early_deliveries_hi                 = 5
late_deliveries_hi                  = 8
avg_prep_vid_time_hi                = 300
largest_order_size_lo               = 2
largest_order_size_hi               = 7
master_classes_attended_hi          = 3
median_meal_rating_hi               = 5
avg_clicks_per_visit_lo             = 8
total_photos_viewed_hi              = 400

# Developing feature columns for outliers

# total_meals_ordered_hi_lo
df['out_total_meals_ordered'] = 0
condition_hi = df.loc[0:,'out_total_meals_ordered'][df['total_meals_ordered'] > total_meals_ordered_hi]
condition_lo = df.loc[0:,'out_total_meals_ordered'][df['total_meals_ordered'] < total_meals_ordered_lo]

df['out_total_meals_ordered'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_total_meals_ordered'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# unique_meals_purch_hi
df['out_unique_meals_purch'] = 0
condition_hi = df.loc[0:,'out_unique_meals_purch'][df['unique_meals_purch'] > unique_meals_purch_hi]

df['out_unique_meals_purch'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# contacts_w_customer_service_hi_lo
df['out_contacts_w_customer_service'] = 0
condition_hi = df.loc[0:,'out_contacts_w_customer_service'][df['contacts_w_customer_service'] > contacts_w_customer_service_hi]
condition_lo = df.loc[0:,'out_contacts_w_customer_service'][df['contacts_w_customer_service'] < contacts_w_customer_service_lo]

df['out_contacts_w_customer_service'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_contacts_w_customer_service'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# product_categories_viewed_hi_lo
df['out_product_categories_viewed'] = 0
condition_hi = df.loc[0:,'out_product_categories_viewed'][df['product_categories_viewed'] > product_categories_viewed_hi]
condition_lo = df.loc[0:,'out_product_categories_viewed'][df['product_categories_viewed'] < product_categories_viewed_lo]

df['out_product_categories_viewed'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_product_categories_viewed'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# avg_time_per_site_visit_hi_lo
df['out_avg_time_per_site_visit'] = 0
condition_hi = df.loc[0:,'out_avg_time_per_site_visit'][df['avg_time_per_site_visit'] > avg_time_per_site_visit_hi]
condition_lo = df.loc[0:,'out_avg_time_per_site_visit'][df['avg_time_per_site_visit'] < avg_time_per_site_visit_lo]

df['out_avg_time_per_site_visit'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)
df['out_avg_time_per_site_visit'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# cancellations_before_noon_hi
df['out_cancellations_before_noon'] = 0
condition_hi = df.loc[0:,'out_cancellations_before_noon'][df['cancellations_before_noon'] > cancellations_before_noon_hi]

df['out_cancellations_before_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# cancellations_after_noon_hi
df['out_cancellations_after_noon'] = 0
condition_hi = df.loc[0:,'out_cancellations_after_noon'][df['cancellations_after_noon'] > cancellations_after_noon_hi]

df['out_cancellations_after_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# mobile_logins_lo
df['out_mobile_logins'] = 0
condition_lo = df.loc[0:,'out_mobile_logins'][df['mobile_logins'] < mobile_logins_lo]

df['out_mobile_logins'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# pc_logins_hi
df['out_pc_logins'] = 0
condition_hi = df.loc[0:,'out_pc_logins'][df['pc_logins'] > pc_logins_hi]

df['out_pc_logins'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# weekly_plan_hi
df['out_weekly_plan'] = 0
condition_hi = df.loc[0:,'out_weekly_plan'][df['weekly_plan'] > weekly_plan_hi]

df['out_weekly_plan'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# early_deliveries_hi
df['out_early_deliveries'] = 0
condition_hi = df.loc[0:,'out_early_deliveries'][df['early_deliveries'] > early_deliveries_hi]

df['out_early_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# late_deliveries_hi
df['out_late_deliveries'] = 0
condition_hi = df.loc[0:,'out_late_deliveries'][df['late_deliveries'] > late_deliveries_hi]

df['out_late_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# avg_prep_vid_time_hi
df['out_avg_prep_vid_time'] = 0
condition_hi = df.loc[0:,'out_avg_prep_vid_time'][df['avg_prep_vid_time'] > avg_prep_vid_time_hi]

df['out_avg_prep_vid_time'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# largest_order_size_hi_lo
df['out_largest_order_size'] = 0
condition_hi = df.loc[0:,'out_largest_order_size'][df['largest_order_size'] > largest_order_size_hi]
condition_lo = df.loc[0:,'out_largest_order_size'][df['largest_order_size'] < largest_order_size_lo]

df['out_largest_order_size'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

df['out_largest_order_size'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# master_classes_attended_hi
df['out_master_classes_attended'] = 0
condition_hi = df.loc[0:,'out_master_classes_attended'][df['master_classes_attended'] > master_classes_attended_hi]

df['out_master_classes_attended'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# median_meal_rating_hi
df['out_median_meal_rating'] = 0
condition_hi = df.loc[0:,'out_median_meal_rating'][df['median_meal_rating'] > median_meal_rating_hi]

df['out_median_meal_rating'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# avg_clicks_per_visit_lo
df['out_avg_clicks_per_visit'] = 0
condition_lo = df.loc[0:,'out_avg_clicks_per_visit'][df['avg_clicks_per_visit'] < avg_clicks_per_visit_lo]

df['out_avg_clicks_per_visit'].replace(to_replace = condition_lo,
                                value      = 1,
                                inplace    = True)

# total_photos_viewed_hi
df['out_total_photos_viewed'] = 0
condition_hi = df.loc[0:,'out_total_photos_viewed'][df['total_photos_viewed'] > total_photos_viewed_hi]

df['out_total_photos_viewed'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# revenue_hi
df['out_revenue'] = 0
condition_hi = df.loc[0:,'out_revenue'][df['revenue'] > revenue_hi]

df['out_revenue'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


#setting trend based thresholds
total_meals_ordered_lo                     = 20 # minimum of 11 to even be included in the dataset
unique_meals_purch_change_hi               = 9  # different data structure above
contacts_w_customer_service_change_hi      = 10 # trend changes after
avg_time_per_site_visit_change_hi          = 250
weekly_plan_hi                             = 15
avg_prep_vid_time_hi                       = 320
largest_order_size_change_hi               = 6  # trend changes after this point
median_meal_rating_change_lo               = 3  # trend changes after this point
avg_clicks_per_visit_change_hi             = 10 # trend changes after this point
avg_clicks_per_visit_change_lo             = 8  # no revenue blow this point
total_photos_viewed_change_at              = 0  # zero inflated

# developing features (columns) for trend changes

#change above threshold

# unique_meals_purch_change_hi
df['change_unique_meals_purch'] = 0
condition = df.loc[0:,'change_unique_meals_purch'][df['unique_meals_purch'] > unique_meals_purch_change_hi]

df['change_unique_meals_purch'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# contacts_w_customer_service_change_hi
df['change_contacts_w_customer_service'] = 0
condition = df.loc[0:,'change_contacts_w_customer_service'][df['contacts_w_customer_service'] > contacts_w_customer_service_change_hi]

df['change_contacts_w_customer_service'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# avg_time_per_site_visit_change_hi
df['change_avg_time_per_site_visit'] = 0
condition = df.loc[0:,'change_avg_time_per_site_visit'][df['avg_time_per_site_visit'] > avg_time_per_site_visit_change_hi]

df['change_avg_time_per_site_visit'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# largest_order_size_change_hi
df['change_largest_order_size'] = 0
condition = df.loc[0:,'change_largest_order_size'][df['largest_order_size'] > largest_order_size_change_hi]

df['change_largest_order_size'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# avg_clicks_per_visit_change_hi
df['change_avg_clicks_per_visit'] = 0
condition = df.loc[0:,'change_avg_clicks_per_visit'][df['avg_clicks_per_visit'] > avg_clicks_per_visit_change_hi]

df['change_avg_clicks_per_visit'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# weekly_plan_hi
df['change_weekly_plan'] = 0
condition = df.loc[0:,'change_weekly_plan'][df['weekly_plan'] > weekly_plan_hi]

df['change_weekly_plan'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)



# Change at threshold 

# total_photos_viewed_change_at
df['change_total_photos_viewed'] = 0
condition = df.loc[0:,'change_total_photos_viewed'][df['total_photos_viewed'] == total_photos_viewed_change_at]

df['change_total_photos_viewed'].replace(to_replace = condition,
                                       value      = 1,
                                       inplace    = True)

# Change below threshold 

# total_meals_ordered_lo
df['change_total_meals_ordered'] = 0
condition = df.loc[0:,'change_total_meals_ordered'][df['total_meals_ordered'] < total_meals_ordered_lo]

df['change_total_meals_ordered'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# median_meal_rating_change_lo
df['change_median_meal_rating'] = 0
condition = df.loc[0:,'change_median_meal_rating'][df['median_meal_rating'] < median_meal_rating_change_lo]

df['change_median_meal_rating'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# avg_clicks_per_visit_change_lo
df['change_lo_avg_clicks_per_visit_change'] = 0
condition = df.loc[0:,'change_lo_avg_clicks_per_visit_change'][df['avg_clicks_per_visit'] < avg_clicks_per_visit_change_lo]

df['change_lo_avg_clicks_per_visit_change'].replace(to_replace = condition,
                                   value      = 1,
                                   inplace    = True)

# Splitting emails to categorise them

# STEP 1: splitting personal emails

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in df.iterrows():
    
    # splitting email domain at '@'
    split_email = df.loc[index, 'email'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# STEP 2: concatenating with original DataFrame

# renaming column to concatenate
email_df.columns = ['0' , 'email_domain']


# concatenating email_domain with df DataFrame
df = pd.concat([df, email_df['email_domain']],
                     axis = 1)

# aggreate emails

# email domain types
personal_email_domains = [  '@gmail.com',
                            '@yahoo.com',
                            '@protonmail.com']
professional_email_domains = ['@mmm.com',
                            '@amex.com',
                            '@apple.com',
                            '@boeing.com',
                            '@caterpillar.com',
                            '@chevron.com',
                            '@cisco.com',
                            '@cocacola.com',
                            '@disney.com',
                            '@dupont.com',
                            '@exxon.com',
                            '@ge.org',
                            '@goldmansacs.com',
                            '@homedepot.com',
                            '@ibm.com',
                            '@intel.com',
                            '@jnj.com',
                            '@jpmorgan.com',
                            '@mcdonalds.com',
                            '@merck.com',
                            '@microsoft.com',
                            '@nike.com',
                            '@pfizer.com',
                            '@pg.com',
                            '@travelers.com',
                            '@unitedtech.com',
                            '@unitedhealth.com',
                            '@verizon.com',
                            '@visa.com',
                            '@walmart.com']

junk_email_domains  = [     '@me.com',
                            '@aol.com',
                            '@hotmail.com',
                            '@live.com',
                            '@msn.com',
                            '@passport.com']


# placeholder list
placeholder_lst = []


# looping to group observations by domain type
for domain in df['email_domain']:
    
    if '@' + domain in personal_email_domains:
        placeholder_lst.append('personal')
        

    elif '@' + domain in professional_email_domains:
        placeholder_lst.append('professional')
        
        
    elif '@' + domain in junk_email_domains:
        placeholder_lst.append('junk')


    else:
            print('Unknown')


# concatenating with original DataFrame
df['domain_group'] = pd.Series(placeholder_lst)

# creating dummy variables for email domain groups
one_hot_domain_group = pd.get_dummies(df['domain_group'])

# joining codings together
df = df.join([one_hot_domain_group])

#creating the log of revenue
df['log_revenue']= np.log(df['revenue'])

# calculating if they bought any beverages.
# maximum price is 23, so everything above will go to beverages.
# we have to reverse engineer this metric, but it should be available.
# added safeguard for new customers without revenue

df['avg_spend'] = df['revenue']/df['total_meals_ordered']
avg_spend_change_hi = 23
df['beverages'] = 0
condition = df.loc[0:,'beverages'][df['avg_spend'] > avg_spend_change_hi]
condition_0 = df.loc[0:,'beverages'][df['avg_spend'] == 0]

df['beverages'].replace(to_replace = condition,
                                      value      = 1,
                                      inplace    = True)
df['beverages'].replace(to_replace = condition_0,
                                      value      = 0,
                                      inplace    = True)
df=df.drop('avg_spend',axis=1)



################################################################################
# Train/Test Split
################################################################################

# use this space to set up testing and validation sets using train/test split

# Note: Be sure to set test_size = 0.25

# selecting x variables for this model
x_variables = ['cross_sell_success',
       'total_meals_ordered', 
       'unique_meals_purch',
       'contacts_w_customer_service',
       'avg_time_per_site_visit',
       'cancellations_after_noon',
       'package_locker',
       'avg_prep_vid_time', 
       'largest_order_size', 
       'master_classes_attended',
       'median_meal_rating',
       'out_total_meals_ordered',
       'out_pc_logins',
       'out_avg_prep_vid_time', 
       'out_largest_order_size',
       'out_master_classes_attended',
       'out_avg_clicks_per_visit',
       'change_contacts_w_customer_service',
       'change_avg_clicks_per_visit',
       'change_total_photos_viewed', 
       'change_total_meals_ordered',
       'change_median_meal_rating', 
       'change_lo_avg_clicks_per_visit_change',
       'junk', 
       'personal', 
       'professional',
       'beverages']


# Preparing a DataFrame based the the analysis above
df_data   = df.loc[ : , x_variables]


# Preparing the target variable
df_target = df.loc[:, 'log_revenue']


# running train/test split again
X_train, X_test, y_train, y_test = train_test_split(
            df_data,
            df_target,
            test_size = 0.25,
            random_state = 222)




################################################################################
# Final Model (instantiate, fit, and predict)
################################################################################

# use this space to instantiate, fit, and predict on your final model

# initiating
GB = GradientBoostingRegressor(
                               max_depth= 2, 
                               min_samples_leaf=0.08, 
                               n_estimators= 500, 
                               random_state=222)


# fitting the model
GB.fit(X_train, y_train)


# predicting
GB_pred = GB.predict(X_test)


# SCORING the results
print('Gradient Boosting Training Score:', GB.score(X_train, y_train).round(4))
print('Gradient Boosting Testing Score:',  GB.score(X_test, y_test).round(4))


# saving scoring data
GB_score_train = GB.score(X_train, y_train).round(4)
GB_score_test  = GB.score(X_test, y_test).round(4)




################################################################################
# Final Model Score (score)
################################################################################

# use this space to score your final model on the testing set
# MAKE SURE TO SAVE YOUR TEST SCORE AS test_score
# Example: test_score = final_model.score(X_test, y_test)

test_score = GB_score_test

"""

 

# calculating execution time
elapsed_time = timeit.timeit(code_to_test, number=3)/3
print(elapsed_time)




Gradient Boosting Training Score: 0.8981
Gradient Boosting Testing Score: 0.8427
Gradient Boosting Training Score: 0.8981
Gradient Boosting Testing Score: 0.8427
